An example of how to calculate time series of seasonal averages (DJF1, MAM1, JJA1, SON1, DJF2, MAM2,...) from a time series of monthly grids using the rolling object in xarray.

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import xarray as xr
import numpy as np
import pandas as pd

## Generate a test data set

In [29]:
da = xr.DataArray(np.random.rand(12,3,5), 
                  coords=[pd.date_range('1/1/1999', periods=12, freq=pd.DateOffset(months=1)),
                          np.arange(3), np.arange(5)], 
                  dims=['time','x','y'])

da

<xarray.DataArray (time: 12, x: 3, y: 5)>
array([[[0.35977 , 0.565422, 0.210938, 0.792952, 0.085355],
        [0.187973, 0.06471 , 0.226647, 0.790178, 0.571173],
        [0.124212, 0.102916, 0.053614, 0.610464, 0.880025]],

       [[0.545342, 0.293587, 0.563517, 0.460485, 0.587294],
        [0.992031, 0.977436, 0.051096, 0.078986, 0.282043],
        [0.11573 , 0.432967, 0.985252, 0.010901, 0.27323 ]],

       [[0.533764, 0.546878, 0.943126, 0.29923 , 0.993986],
        [0.963265, 0.534068, 0.761181, 0.639284, 0.381543],
        [0.742412, 0.118085, 0.1644  , 0.090871, 0.521005]],

       [[0.023849, 0.482714, 0.526002, 0.7694  , 0.791445],
        [0.867117, 0.31588 , 0.784228, 0.765097, 0.689733],
        [0.730251, 0.090332, 0.624075, 0.410771, 0.217034]],

       [[0.634921, 0.259869, 0.408732, 0.1045  , 0.372487],
        [0.608734, 0.366524, 0.292194, 0.553235, 0.936964],
        [0.441155, 0.331004, 0.889506, 0.908125, 0.380662]],

       [[0.918451, 0.931375, 0.349117, 0.632912,

## Generate a rolling object with stride 3
This assumes the monthly time series starts in January.  The (time=3) Window sets up the aggregator to do seasonal averages.  center=True assigns the time of the center month to the aggregated values (e.g. Jan 1 is assigned to DJF).

In [52]:
r = da.rolling(time=3, center=True)
r

DataArrayRolling [window->3,center->True,dim->time]

## Calculate seasonal means
The rolling object aggregates DJF, JFM, FMA, MAM,....  For standard climatological seasons I only want every 3rd average, so I set stride=3.  The mean is then calculated over the window_dim, which is time in this case. 

In [59]:
season = r.construct('window_dim', stride=3).mean(dim='window_dim')
season

<xarray.DataArray (time: 4, x: 3, y: 5)>
array([[[0.452556, 0.429505, 0.387227, 0.626718, 0.336325],
        [0.590002, 0.521073, 0.138871, 0.434582, 0.426608],
        [0.119971, 0.267942, 0.519433, 0.310682, 0.576627]],

       [[0.397511, 0.42982 , 0.625953, 0.391043, 0.719306],
        [0.813039, 0.405491, 0.612534, 0.652539, 0.669413],
        [0.637939, 0.179807, 0.559327, 0.469922, 0.3729  ]],

       [[0.652818, 0.544797, 0.662618, 0.81424 , 0.384133],
        [0.2519  , 0.284903, 0.554429, 0.494923, 0.248229],
        [0.332041, 0.539491, 0.7057  , 0.172409, 0.608645]],

       [[0.486088, 0.478018, 0.154543, 0.076383, 0.458487],
        [0.416691, 0.683423, 0.874394, 0.854409, 0.086185],
        [0.487375, 0.362118, 0.434939, 0.602895, 0.536056]]])
Coordinates:
  * time     (time) datetime64[ns] 1999-01-01 1999-04-01 1999-07-01 1999-10-01
  * x        (x) int64 0 1 2
  * y        (y) int64 0 1 2 3 4

We can do operations on the seasonal averages using groupby

In [44]:
#fug = season.set_index(year = season.time.dt.year, append=True)
season.groupby('time.season').groups

{'DJF': [0], 'JJA': [2], 'MAM': [1], 'SON': [3]}